In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('invoice.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [3]:
text

'id\ttext\ttag\n000.jpg\tCORPORATE\tO\n000.jpg\tPACKAGING\tO\n000.jpg\t&\tO\n000.jpg\tPROMOTIONS\tO\n000.jpg\tBill\tO\n000.jpg\tof\tO\n000.jpg\tLading\tO\n000.jpg\tTo\tO\n000.jpg\tConsignee\tO\n000.jpg\t—\tO\n000.jpg\tSAM\'S\tB-CUSTOMER_NAME\n000.jpg\tDIST.CTR\tI-CUSTOMER_NAME\n000.jpg\t#6493\tI-CUSTOMER_NAME\n000.jpg\tStreet\tB-CUSTOMER_ADDRESS\n000.jpg\t1000\tI-CUSTOMER_ADDRESS\n000.jpg\tS.\tI-CUSTOMER_ADDRESS\n000.jpg\tCUCAMONGA\tI-CUSTOMER_ADDRESS\n000.jpg\tAVE\tI-CUSTOMER_ADDRESS\n000.jpg\t  \tO\n000.jpg\t  \tO\n000.jpg\tBILL\tB-INVOICE_NO\n000.jpg\tOF\tI-INVOICE_NO\n000.jpg\tLADING\tI-INVOICE_NO\n000.jpg\tNUMBER:\tI-INVOICE_NO\n000.jpg\t8175105\tI-INVOICE_NO\n000.jpg\tPO#\tO\n000.jpg\tBILL\tO\n000.jpg\tDATE:\tO\n000.jpg\tFROM\tO\n000.jpg\tAR50\tB-COMPANY_NAME\n000.jpg\tCORPORATE\tI-COMPANY_NAME\n000.jpg\tPACKAGING\tI-COMPANY_NAME\n000.jpg\t   \tO\n000.jpg\tShipper\tO\n000.jpg\t  \tO\n000.jpg\t \tO\n000.jpg\t    \tO\n000.jpg\t \tO\n000.jpg\tDestination\tO\n000.jpg\t \tO\n000.jpg\t

In [4]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [5]:
df = pd.DataFrame(data[1:],columns=data[0])

In [6]:
df.head(10)

,id,text,tag
0,000.jpg,CORPORATE,O
1,000.jpg,PACKAGING,O
2,000.jpg,&,O
3,000.jpg,PROMOTIONS,O
4,000.jpg,Bill,O
5,000.jpg,of,O
6,000.jpg,Lading,O
7,000.jpg,To,O
8,000.jpg,Consignee,O
9,000.jpg,—,O


In [7]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [8]:
df['text'] = df['text'].apply(cleanText)

In [9]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

/tmp/ipykernel_3741/3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [10]:
dataClean.head(10)

,id,text,tag
0,000.jpg,corporate,O
1,000.jpg,packaging,O
3,000.jpg,promotions,O
4,000.jpg,bill,O
5,000.jpg,of,O
6,000.jpg,lading,O
7,000.jpg,to,O
8,000.jpg,consignee,O
9,000.jpg,—,O
10,000.jpg,sams,B-CUSTOMER_NAME


In [12]:
group = dataClean.groupby(by='id')

In [13]:
cards = group.groups.keys()

In [15]:
allCardsData = []
for card in cards:
    cardData = []
    grouparray = group.get_group(card)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    cardData = (content,annotations)
    allCardsData.append(cardData)

In [16]:
allCardsData

[('corporate packaging promotions bill of lading to consignee — sams dist.ctr 6493 street 1000 s. cucamonga ave bill of lading number 8175105 po bill date from ar50 corporate packaging shipper destination city/state/zip "ontario," ca 91761 third party "billing," send to name arizona beverages llc c ompany arizona beverages llc street city/state/zip street 1555 s. archibald avenue origin city/state/zip "ontario," ca 91761 special instructions shippers instructions no. shipping cases no. shipping pallets description of articles special marks exceptions equip 317 7 equip arivat 12/20/19 459 carrier bawb ° seal 3289785 remit c.0.d. address reseal door/zone del date appointment 12/20/19 500 have read and understan appointment / drop rules note w here the rate is depende on "value," shippers are required to state specifically in writing the agre or declared value of the property. 7 agreed or declared value of the driver signature property is hereby specifically state by the shipper to be not

In [17]:
import random

In [18]:
random.shuffle(allCardsData)

In [19]:
len(allCardsData)

1

In [20]:
TrainData = allCardsData[:1]
TestData = allCardsData[1:]

In [21]:
import pickle

In [22]:
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))